In [62]:
# scatter plot + box plot (& istogram) per ogni combinazione di features

# notebook:
# Legge dalla directory out/ tutti i file che ci sono (uno per feature) e mettere insieme tutto in uno stesso file
# (sicuramente la run di twitter andra' da sola e avra' un file di out a parte: si vuole unire ai file gia' presenti delle altre features:
# pandas.join... Metti tutto in un file solo, e poi fai i vari plot.

import os, pandas

# files from "features_exploration" out (.csv)
out_path = ".." + os.sep + "out" + os.sep
dataframes = []
features = []

# reads file (= features names = column name)
for result_file in os.listdir(out_path):
    features.append(result_file)
    dataframes.append(pandas.read_csv(out_path + result_file))

# directory not empty check
if len(dataframes) == 0:
    print("No csv files in " + out_path)
    exit(1)
    
# joins all data!
final_data = dataframes[0]
for dataframe in dataframes[1:]:
    final_data = pandas.merge(final_data, dataframe, on="id", how="outer")
    
# picks only id, label and features columns (merge produces junk too)
columns = ["id", "label"]
columns.extend(features)

print(final_data[columns])

          id     label  morfological_complexity  twittersearch  \
0          1  not_fake                     23.0            1.0   
1          2      fake                     28.0          100.0   
2          3      fake                     22.0          100.0   
3          4      fake                     28.0            6.0   
4          5  not_fake                     19.0            0.0   
5          6      fake                     27.0          100.0   
6          7  not_fake                     25.0            0.0   
7          8  not_fake                     29.0           11.0   
8          9  not_fake                     21.0            9.0   
9         10  not_fake                     24.0           14.0   
10        11  not_fake                     22.0            1.0   
11        12      fake                     17.0            0.0   
12        13      fake                     35.0           33.0   
13        14      fake                     25.0          100.0   
14        